In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

pd.set_option('display.max_columns', None)

In [178]:
agent_data = pd.read_csv('Data/datastorm_agent_data.csv')
policy_data = pd.read_csv("Data/datastorm_policy_data_short.csv")

In [179]:
policy_data

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_entry_age,spouse_smoker_flag,total_sum_assuared,premium_value,rider1_prem,rider1_sum_assuared,rider2_prem,rider2_sum_assuared,rider3_prem,rider3_sum_assuared,rider4_prem,rider4_sum_assuared,rider5_prem,rider5_sum_assuared,rider6_prem,rider6_sum_assuared,rider7_prem,rider7_sum_assuared,rider8_prem,rider8_sum_assuared,rider9_prem,rider9_sum_assuared,rider10_prem,rider10_sum_assuared,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20190131,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
1,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20200301,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
2,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
3,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20191001,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
4,226.0,12,M,INFORCE,12/28/2018,3/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20190228,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,7/7/2005,1/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20181231,20200701,C100533,AG106849,RE001,RETIREMENT,POL25553
9832,200.0,30,Q,INFORCE,7/7/2005,4/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200201,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9833,200.0,30,Q,INFORCE,7/7/2005,10/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20191001,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9834,200.0,30,Q,INFORCE,7/7/2005,1/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200101,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553


In [180]:
snapshot_months = np.sort(pd.unique(policy_data["policy_snapshot_as_on"]))

# def divide_chunks(l, n):
#     # Divide months into k bins
#     for i in range(0, len(l), n): 
#         yield l[i:i + n]

# k = 6
# snapshot_months = list(divide_chunks(snapshot_months, k))

snapshot_months

array([20181231, 20190131, 20190228, 20190331, 20190430, 20190531,
       20190630, 20190731, 20190831, 20190901, 20191001, 20191101,
       20191201, 20200101, 20200201, 20200301, 20200401, 20200501,
       20200601, 20200701], dtype=int64)

In [181]:
valid_policy_data = policy_data[policy_data["termination_dt"].isnull()]
valid_policy_data

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,main_holder_entry_age,main_holder_smoker_flag,spouse_gender,spouse_dob,spouse_entry_age,spouse_smoker_flag,total_sum_assuared,premium_value,rider1_prem,rider1_sum_assuared,rider2_prem,rider2_sum_assuared,rider3_prem,rider3_sum_assuared,rider4_prem,rider4_sum_assuared,rider5_prem,rider5_sum_assuared,rider6_prem,rider6_sum_assuared,rider7_prem,rider7_sum_assuared,rider8_prem,rider8_sum_assuared,rider9_prem,rider9_sum_assuared,rider10_prem,rider10_sum_assuared,child1_dob,child1_gender,child2_dob,child2_gender,child3_dob,child3_gender,child4_dob,child4_gender,child5_dob,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20190131,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
1,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20200301,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
2,226.0,12,M,INFORCE,12/28/2018,1/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,FLOUR & GRAIN MILL W,20181231,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
3,226.0,12,M,LAPSED,12/28/2018,4/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20191001,20200701,C100000,AG106669,IN001,INVESTMENT,POL29331
4,226.0,12,M,INFORCE,12/28/2018,3/28/2019,NaN,NaN,M,1/1/1991,28,N,NaN,NaN,NaN,NaN,275000,60144,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEQUE,FLOUR & GRAIN MILL W,20190228,20200701,C100000,AG101392,IN001,INVESTMENT,POL29331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9831,200.0,30,Q,INFORCE,7/7/2005,1/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20181231,20200701,C100533,AG106849,RE001,RETIREMENT,POL25553
9832,200.0,30,Q,INFORCE,7/7/2005,4/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200201,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9833,200.0,30,Q,INFORCE,7/7/2005,10/7/2019,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20191001,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553
9834,200.0,30,Q,INFORCE,7/7/2005,1/7/2020,NaN,NaN,F,10/1/1965,40,N,M,12/1/1960,45.0,N,500000,26808,0,0,1752,1000000,0,0,0,0,0,0,0,0,376,500000,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CASH,NaN,20200101,20200701,C100533,AG100338,RE001,RETIREMENT,POL25553


In [182]:
# Useful Columns
columns = ["client_code", "main_holder_occupation_cd", "policy_term", "policy_payment_mode", "policy_status", "main_holder_gender", "main_holder_entry_age", "policy_snapshot_as_on" ,"main_holder_smoker_flag", "total_sum_assuared", "payment_method" ]

In [183]:
# Return cross sell status for each month
def is_cross_sell(no_of_policies):
    cross_sell = [];

    for k in range(len(no_of_policies)):
        n = 6
        if(k + n) >= len(no_of_policies) :
            n = len(no_of_policies) - k -1
        if(no_of_policies[k + n] > no_of_policies[k]):
            cross_sell.append(True)
        else:
            cross_sell.append(False)
    return cross_sell

new_data_set = pd.DataFrame(columns= columns + ["cross_sell"])
test_data = pd.DataFrame(columns= columns + ["cross_sell"])

for client_name, client_data in valid_policy_data.groupby(["client_code"]):
    no_of_policies = client_data["policy_snapshot_as_on"].value_counts().sort_index()

    # Get only one data row for each month
    df = client_data.drop_duplicates("policy_snapshot_as_on", keep='last')
    df = df.filter(columns)
    df["cross_sell"] = is_cross_sell(list(no_of_policies))

    new_data_set = new_data_set.append(df)

    test_data = test_data.append(df[df["policy_snapshot_as_on"] == 20200701])

test_data.drop("cross_sell", inplace=True, axis=1)
new_data_set



,client_code,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,main_holder_gender,main_holder_entry_age,policy_snapshot_as_on,main_holder_smoker_flag,total_sum_assuared,payment_method,cross_sell
0,C100000,226.0,12,M,INFORCE,M,28,20190131,N,275000,CASH,False
1,C100000,226.0,12,M,LAPSED,M,28,20200301,N,275000,CHEQUE,False
2,C100000,226.0,12,M,INFORCE,M,28,20181231,N,275000,CASH,False
3,C100000,226.0,12,M,LAPSED,M,28,20191001,N,275000,CHEQUE,False
4,C100000,226.0,12,M,INFORCE,M,28,20190228,N,275000,CHEQUE,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9831,C100533,200.0,30,Q,INFORCE,F,40,20181231,N,500000,CASH,False
9832,C100533,200.0,30,Q,INFORCE,F,40,20200201,N,500000,CASH,False
9833,C100533,200.0,30,Q,INFORCE,F,40,20191001,N,500000,CASH,False
9834,C100533,200.0,30,Q,INFORCE,F,40,20200101,N,500000,CASH,False


In [184]:
new_data_set

,client_code,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,main_holder_gender,main_holder_entry_age,policy_snapshot_as_on,main_holder_smoker_flag,total_sum_assuared,payment_method,cross_sell
0,C100000,226.0,12,M,INFORCE,M,28,20190131,N,275000,CASH,False
1,C100000,226.0,12,M,LAPSED,M,28,20200301,N,275000,CHEQUE,False
2,C100000,226.0,12,M,INFORCE,M,28,20181231,N,275000,CASH,False
3,C100000,226.0,12,M,LAPSED,M,28,20191001,N,275000,CHEQUE,False
4,C100000,226.0,12,M,INFORCE,M,28,20190228,N,275000,CHEQUE,False
...,...,...,...,...,...,...,...,...,...,...,...,...
9831,C100533,200.0,30,Q,INFORCE,F,40,20181231,N,500000,CASH,False
9832,C100533,200.0,30,Q,INFORCE,F,40,20200201,N,500000,CASH,False
9833,C100533,200.0,30,Q,INFORCE,F,40,20191001,N,500000,CASH,False
9834,C100533,200.0,30,Q,INFORCE,F,40,20200101,N,500000,CASH,False


In [185]:

new_data_set.to_csv("Data/datastorm_policy_data_formatted.csv", index=False)
test_data.to_csv("Data/formatted_test_set.csv", index=False)

In [194]:
# categorical data
cat_data = ["main_holder_occupation_cd", "policy_payment_mode", "policy_status", "main_holder_gender", "main_holder_smoker_flag", "payment_method"]

drop_data = ["client_code"]
columns = []

for element in drop_data:
    if element in cat_data:
        cat_data.remove(element)

def format_data(data_set, is_test=False):
    f_data_set = pd.get_dummies(data_set, prefix=cat_data, columns=cat_data)
    f_data_set = f_data_set.drop(drop_data, axis = 1)
    if(is_test):
        x = f_data_set
        return x
    else:
        x = f_data_set.drop("cross_sell", axis = 1)
        y = f_data_set["cross_sell"]
        global columns
        columns = x.columns
        return x, np.array(y).astype(int)

x_tot, y_tot = format_data(new_data_set)
x_test = format_data(test_data, is_test=True)

x_tot, x_test = x_tot.align(x_test, join="outer", axis=1, fill_value=0)

x_train, x_val, y_train, y_val = train_test_split(x_tot, y_tot, test_size=0.2, random_state=0)

print(len(x_train), len(x_val), len(x_test))
print(x_tot.shape, len(columns))

7271 1818 446
(9089, 133) 133


In [195]:
pd.DataFrame(x_test, columns=columns)

,policy_term,main_holder_entry_age,policy_snapshot_as_on,total_sum_assuared,main_holder_occupation_cd_3.0,main_holder_occupation_cd_4.0,main_holder_occupation_cd_9.0,main_holder_occupation_cd_12.0,main_holder_occupation_cd_48.0,main_holder_occupation_cd_51.0,main_holder_occupation_cd_52.0,main_holder_occupation_cd_53.0,main_holder_occupation_cd_54.0,main_holder_occupation_cd_62.0,main_holder_occupation_cd_63.0,main_holder_occupation_cd_66.0,main_holder_occupation_cd_71.0,main_holder_occupation_cd_81.0,main_holder_occupation_cd_88.0,main_holder_occupation_cd_92.0,main_holder_occupation_cd_96.0,main_holder_occupation_cd_101.0,main_holder_occupation_cd_103.0,main_holder_occupation_cd_107.0,main_holder_occupation_cd_109.0,main_holder_occupation_cd_116.0,main_holder_occupation_cd_121.0,main_holder_occupation_cd_134.0,main_holder_occupation_cd_136.0,main_holder_occupation_cd_141.0,main_holder_occupation_cd_156.0,main_holder_occupation_cd_157.0,main_holder_occupation_cd_158.0,main_holder_occupation_cd_159.0,main_holder_occupation_cd_160.0,main_holder_occupation_cd_162.0,main_holder_occupation_cd_163.0,main_holder_occupation_cd_164.0,main_holder_occupation_cd_165.0,main_holder_occupation_cd_166.0,main_holder_occupation_cd_167.0,main_holder_occupation_cd_168.0,main_holder_occupation_cd_175.0,main_holder_occupation_cd_177.0,main_holder_occupation_cd_193.0,main_holder_occupation_cd_195.0,main_holder_occupation_cd_200.0,main_holder_occupation_cd_204.0,main_holder_occupation_cd_205.0,main_holder_occupation_cd_208.0,main_holder_occupation_cd_214.0,main_holder_occupation_cd_218.0,main_holder_occupation_cd_220.0,main_holder_occupation_cd_225.0,main_holder_occupation_cd_226.0,main_holder_occupation_cd_233.0,main_holder_occupation_cd_234.0,main_holder_occupation_cd_245.0,main_holder_occupation_cd_247.0,main_holder_occupation_cd_249.0,main_holder_occupation_cd_253.0,main_holder_occupation_cd_261.0,main_holder_occupation_cd_273.0,main_holder_occupation_cd_274.0,main_holder_occupation_cd_275.0,main_holder_occupation_cd_279.0,main_holder_occupation_cd_285.0,main_holder_occupation_cd_286.0,main_holder_occupation_cd_287.0,main_holder_occupation_cd_293.0,main_holder_occupation_cd_298.0,main_holder_occupation_cd_301.0,main_holder_occupation_cd_302.0,main_holder_occupation_cd_308.0,main_holder_occupation_cd_317.0,main_holder_occupation_cd_321.0,main_holder_occupation_cd_323.0,main_holder_occupation_cd_324.0,main_holder_occupation_cd_329.0,main_holder_occupation_cd_331.0,main_holder_occupation_cd_336.0,main_holder_occupation_cd_345.0,main_holder_occupation_cd_347.0,main_holder_occupation_cd_351.0,main_holder_occupation_cd_353.0,main_holder_occupation_cd_354.0,main_holder_occupation_cd_356.0,main_holder_occupation_cd_361.0,main_holder_occupation_cd_363.0,main_holder_occupation_cd_364.0,main_holder_occupation_cd_366.0,main_holder_occupation_cd_371.0,main_holder_occupation_cd_373.0,main_holder_occupation_cd_381.0,main_holder_occupation_cd_386.0,main_holder_occupation_cd_392.0,main_holder_occupation_cd_400.0,main_holder_occupation_cd_415.0,main_holder_occupation_cd_416.0,main_holder_occupation_cd_417.0,main_holder_occupation_cd_418.0,main_holder_occupation_cd_419.0,main_holder_occupation_cd_421.0,main_holder_occupation_cd_425.0,main_holder_occupation_cd_426.0,main_holder_occupation_cd_428.0,main_holder_occupation_cd_429.0,main_holder_occupation_cd_440.0,main_holder_occupation_cd_442.0,main_holder_occupation_cd_444.0,main_holder_occupation_cd_446.0,main_holder_occupation_cd_447.0,main_holder_occupation_cd_448.0,main_holder_occupation_cd_449.0,main_holder_occupation_cd_450.0,main_holder_occupation_cd_451.0,main_holder_occupation_cd_455.0,main_holder_occupation_cd_458.0,main_holder_occupation_cd_468.0,main_holder_occupation_cd_481.0,policy_payment_mode_H,policy_payment_mode_M,policy_payment_mode_Q,policy_payment_mode_S,policy_payment_mode_Y,policy_status_INFORCE,policy_status_LAPSED,main_holder_gender_F,main_holder_gender_M,main_holder_smoker_flag_N,main_holder

In [196]:
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

pd.DataFrame(x_train, columns=columns).head(10)

,policy_term,main_holder_entry_age,policy_snapshot_as_on,total_sum_assuared,main_holder_occupation_cd_3.0,main_holder_occupation_cd_4.0,main_holder_occupation_cd_9.0,main_holder_occupation_cd_12.0,main_holder_occupation_cd_48.0,main_holder_occupation_cd_51.0,main_holder_occupation_cd_52.0,main_holder_occupation_cd_53.0,main_holder_occupation_cd_54.0,main_holder_occupation_cd_62.0,main_holder_occupation_cd_63.0,main_holder_occupation_cd_66.0,main_holder_occupation_cd_71.0,main_holder_occupation_cd_81.0,main_holder_occupation_cd_88.0,main_holder_occupation_cd_92.0,main_holder_occupation_cd_96.0,main_holder_occupation_cd_101.0,main_holder_occupation_cd_103.0,main_holder_occupation_cd_107.0,main_holder_occupation_cd_109.0,main_holder_occupation_cd_116.0,main_holder_occupation_cd_121.0,main_holder_occupation_cd_134.0,main_holder_occupation_cd_136.0,main_holder_occupation_cd_141.0,main_holder_occupation_cd_156.0,main_holder_occupation_cd_157.0,main_holder_occupation_cd_158.0,main_holder_occupation_cd_159.0,main_holder_occupation_cd_160.0,main_holder_occupation_cd_162.0,main_holder_occupation_cd_163.0,main_holder_occupation_cd_164.0,main_holder_occupation_cd_165.0,main_holder_occupation_cd_166.0,main_holder_occupation_cd_167.0,main_holder_occupation_cd_168.0,main_holder_occupation_cd_175.0,main_holder_occupation_cd_177.0,main_holder_occupation_cd_193.0,main_holder_occupation_cd_195.0,main_holder_occupation_cd_200.0,main_holder_occupation_cd_204.0,main_holder_occupation_cd_205.0,main_holder_occupation_cd_208.0,main_holder_occupation_cd_214.0,main_holder_occupation_cd_218.0,main_holder_occupation_cd_220.0,main_holder_occupation_cd_225.0,main_holder_occupation_cd_226.0,main_holder_occupation_cd_233.0,main_holder_occupation_cd_234.0,main_holder_occupation_cd_245.0,main_holder_occupation_cd_247.0,main_holder_occupation_cd_249.0,main_holder_occupation_cd_253.0,main_holder_occupation_cd_261.0,main_holder_occupation_cd_273.0,main_holder_occupation_cd_274.0,main_holder_occupation_cd_275.0,main_holder_occupation_cd_279.0,main_holder_occupation_cd_285.0,main_holder_occupation_cd_286.0,main_holder_occupation_cd_287.0,main_holder_occupation_cd_293.0,main_holder_occupation_cd_298.0,main_holder_occupation_cd_301.0,main_holder_occupation_cd_302.0,main_holder_occupation_cd_308.0,main_holder_occupation_cd_317.0,main_holder_occupation_cd_321.0,main_holder_occupation_cd_323.0,main_holder_occupation_cd_324.0,main_holder_occupation_cd_329.0,main_holder_occupation_cd_331.0,main_holder_occupation_cd_336.0,main_holder_occupation_cd_345.0,main_holder_occupation_cd_347.0,main_holder_occupation_cd_351.0,main_holder_occupation_cd_353.0,main_holder_occupation_cd_354.0,main_holder_occupation_cd_356.0,main_holder_occupation_cd_361.0,main_holder_occupation_cd_363.0,main_holder_occupation_cd_364.0,main_holder_occupation_cd_366.0,main_holder_occupation_cd_371.0,main_holder_occupation_cd_373.0,main_holder_occupation_cd_381.0,main_holder_occupation_cd_386.0,main_holder_occupation_cd_392.0,main_holder_occupation_cd_400.0,main_holder_occupation_cd_415.0,main_holder_occupation_cd_416.0,main_holder_occupation_cd_417.0,main_holder_occupation_cd_418.0,main_holder_occupation_cd_419.0,main_holder_occupation_cd_421.0,main_holder_occupation_cd_425.0,main_holder_occupation_cd_426.0,main_holder_occupation_cd_428.0,main_holder_occupation_cd_429.0,main_holder_occupation_cd_440.0,main_holder_occupation_cd_442.0,main_holder_occupation_cd_444.0,main_holder_occupation_cd_446.0,main_holder_occupation_cd_447.0,main_holder_occupation_cd_448.0,main_holder_occupation_cd_449.0,main_holder_occupation_cd_450.0,main_holder_occupation_cd_451.0,main_holder_occupation_cd_455.0,main_holder_occupation_cd_458.0,main_holder_occupation_cd_468.0,main_holder_occupation_cd_481.0,policy_payment_mode_H,policy_payment_mode_M,policy_payment_mode_Q,policy_payment_mode_S,policy_payment_mode_Y,policy_status_INFORCE,policy_status_LAPSED,main_holder_gender_F,main_holder_gender_M,main_holder_smoker_flag_N,main_holder

In [197]:
y_val.shape

(1818,)

In [198]:
x_val.shape

(1818, 133)

In [199]:
x_train.shape

(7271, 133)

In [202]:

classifier = XGBClassifier(n_estimators=15, max_depth=5)

classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_val)

print('Accuracy =', classifier.score(x_val, y_val))
print('AUC Score =', roc_auc_score(y_val, y_pred))

Accuracy = 0.977997799779978
AUC Score = 0.5470559845559846


In [201]:
# Nullify Terminated Users

terminated_users = policy_data[policy_data["termination_dt"].notna()].client_code.unique()

In [203]:
test_data = pd.read_csv("Data/testset.csv")

# -- THIS IS NOT FINAL --

y_pred = classifier.predict_proba(x_train)[:,1]

test_data.loc[test_data["map_client_cd"].isin(terminated_users) , "probability_of_cross_sell"] = 0

test_data


,map_client_cd,probability_of_cross_sell
0,C100003,0.406901
1,C100004,0.989584
2,C100009,0.339157
3,C100014,0.301586
4,C100015,0.536124
...,...,...
19137,C154448,0.549853
19138,C154449,0.740576
19139,C154450,0.841277
19140,C154454,0.732790


In [204]:
test_data.to_csv("Data/submission.csv")


In [211]:
y_pred = classifier.predict_proba(x_train)[:,1]

y_pred


train_set()

array([0.13221633, 0.11990596, 0.11990596, ..., 0.11990596, 0.12338398,
       0.13221633], dtype=float32)